# Run 1st cycle model - for mCSPC patients

## Initialization

In [1]:
import pandas as pd
import sys 
import scipy as scipy

# Import my own libraries
sys.path.append("./local/")
import myUtils as utils
import initialize as initialize


done: set up models


## Fitting dynamics

In [ ]:
#########################
#Set up data
#########################
print("init...")
dataDfName = "./data/MCC 19367_C1.xlsx"
txCols = ["ADT","Abi","ARSI"]

patientIdDic ={2:'102',3:'103',4:'104',5:'105',6:'106',7:'107',8:'108',9:'109',10:'110',
              11:'111',12:'112',13:'113',14:'114',15:'115',16:'116',17:'117'}

#########################
#Set model and solver
# #########################
modelDic ={1:'modPCaSX'}
modelNum=1
modelName=modelDic[modelNum]

#fitting
nIterations = 1
n_sample = 400 #number of par sets for each patient
n_wins = 10 #number of winners from n_sample to take

################################
# Set and organize patient data
################################
from os.path import exists
outDir = "./output/"
dataDir=dataDfName

colorPalette = {"Testosterone": '#5ACD7D',"T":'#5ACD7D',"Testo": '#5ACD7D',
                "PSA": "#000000","PSAi": "#000000","P": "#000000",
                "S": '#00FDFF',"TumorSize": "#000000",
                "ADT": "#0000FF","Abi": "#000000","ARSI": "#000000"}

######################################################
# Loop through patients and find best fit parameters
###################################################### 
outputCols=['PSAi'] 

fittingSpecs= pd.read_excel('./Parameters/Pars_'+modelName+'.xlsx','Fitting')
[fitting]=initialize.getFittingSpecs(fittingSpecs) 
my_fit=initialize.Fitting2(nIterations,n_sample,n_wins,fitting)

lArray=initialize.defineInitValues(patientIdDic,dataDir,my_fit)  
utils.patient_loop(patientIdDic,dataDir,lArray,modelName,txCols,colorPalette,outputCols,1,my_fit,outDir)#boolTinput


init...


/Users/gallahja/Moffitt Cancer Center Dropbox/Jill Gallaher/MyPapers/InProgress/PCa early progressors/early_progressors_code_FINAL/./local/initialize.py:98: RuntimeWarning: All-NaN axis encountered
  tmpT=np.nanmax(dataDf0['Testosterone'])
/Users/gallahja/Moffitt Cancer Center Dropbox/Jill Gallaher/MyPapers/InProgress/PCa early progressors/early_progressors_code_FINAL/./local/initialize.py:99: RuntimeWarning: All-NaN axis encountered
  if(np.isnan(np.nanmax(dataDf0['Testosterone']))):


102


100%|██████████| 400/400 [01:26<00:00,  4.60it/s]


plotting....


/Users/gallahja/Moffitt Cancer Center Dropbox/Jill Gallaher/MyPapers/InProgress/PCa early progressors/early_progressors_code_FINAL/./local/initialize.py:98: RuntimeWarning: All-NaN axis encountered
  tmpT=np.nanmax(dataDf0['Testosterone'])
/Users/gallahja/Moffitt Cancer Center Dropbox/Jill Gallaher/MyPapers/InProgress/PCa early progressors/early_progressors_code_FINAL/./local/initialize.py:99: RuntimeWarning: All-NaN axis encountered
  if(np.isnan(np.nanmax(dataDf0['Testosterone']))):


103


 88%|████████▊ | 354/400 [03:12<00:25,  1.78it/s]

## model fit visualization - all patients

In [ ]:
import os
import numpy as np
import models as mods
import odeFuns as odeFuns
import analysisFuns as aFuns
import matplotlib.pyplot as plt

patientIdDic ={2:'102',3:'103',4:'104',5:'105',6:'106',7:'107',8:'108',9:'109',10:'110',
              11:'111',12:'112',13:'113',14:'114',15:'115',16:'116',17:'117'}

solver_kws={'scaleTumourVolume':True, 'method':'DOP853', 'absErr':1.0e-6, 'relErr':1.0e-6, 'suppressOutputB':True}
cols=['r','k','k','k','b','b','k','r','k','r','k','k','k','k','b','k']
#patientIdDic={patIdNum:patientIdDic[patIdNum]}
boolTinput=True

#########################
#Set model and solver
# #########################

fittingSpecs= pd.read_excel('./Parameters/Pars_'+modelName+'.xlsx','Fitting')
[fitting]=initialize.getFittingSpecs(fittingSpecs) 
my_fit=initialize.Fitting2(1,1,1,fitting)
[fullDictionary,dictionary]=initialize.loadFullDictionary(modelName)#parDic
[model,outputFunction]=mods.pickModel(modelName)
modelSpecs= pd.read_excel('./Parameters/Pars_'+modelName+'.xlsx','modelSpecs')
[stateVarsNames,stateVarsType,pops0Input,pops0Output,pops0Type]=initialize.getModelSpecs(modelSpecs) 

varsV=[] 
varsC=[]
for name in fullDictionary.Name:
    entry=fullDictionary[fullDictionary.Name==name]
    if(entry.Type.iloc[0]=='V'):
        varsV.append(name)
    else:
        varsC.append(name)
varsNames=varsC+varsV

tmpModel = odeFuns.setupModel(stateVarsNames,stateVarsType,pops0Input,pops0Output,pops0Type,txCols,model,modelName,colorPalette,outputCols,outputFunction,dt=1)

#################
# Plot PSA
#################
fig, ax = plt.subplots(1, 1, figsize=(8,6))
for pi, patInd in enumerate(patientIdDic):
    patientId = patientIdDic[patInd]

    dataDf = initialize.loadData(dataDfName,patientId)
    lArray=initialize.defineInitValues(patientIdDic,dataDir,my_fit)
    tx = utils.ExtractTxsFromDf(dataDf,txCols=txCols)

    parsFile=os.path.join(outDir,modelName,'parWins'+modelName+patientId+'.xlsx')
    tmp=pd.read_excel(parsFile)
    parDicAdd={'r_S':tmp['r_S'].iloc[0],'X0':tmp['X0'].iloc[0]}

    testo=[]
    if(boolTinput):
        nanT=np.isnan(dataDf['Testosterone'])
        if(nanT[0]):
            dataDf.loc[0,'Testosterone']=np.nanmax(dataDf['Testosterone'])
            nanT[0]=False
        dataDf['Tlog']=np.log10(dataDf['Testosterone'])
        f1T = scipy.interpolate.interp1d(dataDf.Days[~nanT],dataDf['Tlog'][~nanT],fill_value="extrapolate",kind="linear")#"extrapolate"
        teval=range(0,dataDf['Days'][len(dataDf['Days'])-1])
        testo=f1T(teval)
        testo=10**testo
        i=0
        tx=[]
        for d in teval:
            tx.append([d,d+1,dataDf['ADT'].loc[i],dataDf['Abi'].loc[i],dataDf['ARSI'].loc[i]])
            if(d>=dataDf['Days'].loc[i+1]):
                i=i+1 

    topDf=aFuns.runSim(tmpModel,patientId,pi,lArray,parDicAdd,tx,my_fit,boolTinput,testo,solver_kws)
    tMax=290

    if(cols[pi]=='r'):lw=6
    elif(cols[pi]=='b'):lw=4
    else:lw=2

    plt.plot(topDf['Days']/topDf['Days'].iloc[-1],topDf['PSAi'],c=cols[pi],linewidth=lw)
    plt.scatter(dataDf['Days']/dataDf['Days'].iloc[-1],dataDf['PSA'],c=cols[pi])
ax.set(yscale='log')

#################
# Plot PSA/T
#################
fig, ax = plt.subplots(1, 1, figsize=(8,6))
for pi, patInd in enumerate(patientIdDic):
    patientId = patientIdDic[patInd]

    dataDf = initialize.loadData(dataDfName,patientId)
    lArray=initialize.defineInitValues(patientIdDic,dataDir,my_fit)
    tx = utils.ExtractTxsFromDf(dataDf,txCols=txCols)

    parsFile=os.path.join(outDir,modelName,'parWins'+modelName+patientId+'.xlsx')
    tmp=pd.read_excel(parsFile)
    parDicAdd={'r_S':tmp['r_S'].iloc[0],'X0':tmp['X0'].iloc[0]}

    testo=[]
    if(boolTinput):
        nanT=np.isnan(dataDf['Testosterone'])
        if(nanT[0]):
            dataDf.loc[0,'Testosterone']=np.nanmax(dataDf['Testosterone'])
            nanT[0]=False
        dataDf['Tlog']=np.log10(dataDf['Testosterone'])
        f1T = scipy.interpolate.interp1d(dataDf.Days[~nanT],dataDf['Tlog'][~nanT],fill_value="extrapolate",kind="linear")#"extrapolate"
        teval=range(0,dataDf['Days'][len(dataDf['Days'])-1])
        testo=f1T(teval)
        testo=10**testo
        i=0
        tx=[]
        for d in teval:
            tx.append([d,d+1,dataDf['ADT'].loc[i],dataDf['Abi'].loc[i],dataDf['ARSI'].loc[i]])
            if(d>=dataDf['Days'].loc[i+1]):
                i=i+1 

    topDf=aFuns.runSim(tmpModel,patientId,pi,lArray,parDicAdd,tx,my_fit,boolTinput,testo,solver_kws)
    tMax=290

    if(cols[pi]=='r'):lw=6
    elif(cols[pi]=='b'):lw=4
    else:lw=2

    plt.plot(topDf['Days']/topDf['Days'].iloc[-1],topDf['PSAi']/topDf['T'],c=cols[pi],linewidth=lw)
    plt.scatter(dataDf['Days']/dataDf['Days'].iloc[-1],dataDf['PSA']/dataDf['Testosterone'],c=cols[pi])
ax.set(yscale='log')


In [ ]:
thisPlot='PT'#'PSA'#'PT'
thisPat='105'#'105'#'109

#########################
#Set up data
#########################
import seaborn as sns
sns.set(style="white",
        font_scale=1.5,
        rc={'figure.figsize':(10,4)})

#dataDir=dataDfName
patientIdDic ={2:'102',3:'103',4:'104',5:'105',6:'106',7:'107',8:'108',9:'109',10:'110',
              11:'111',12:'112',13:'113',14:'114',15:'115',16:'116',17:'117'}

if(thisPat=='105'):
    patIdNum=5
    pi=3
else:
    patIdNum=9
    pi=7

patInd=0
patientIdDic={patIdNum:patientIdDic[patIdNum]}

#########################
#Setup model
# #########################

modelSpecs= pd.read_excel('./Parameters/Pars_'+modelName+'.xlsx','modelSpecs')
[stateVarsNames,stateVarsType,pops0Input,pops0Output,pops0Type]=initialize.getModelSpecs(modelSpecs) 

outputCols=['PSAi'] 
outDir = "./output/"#+modelName

patientId = list(patientIdDic.values())[patInd]
dataDf = initialize.loadData(dataDfName,patientId)
lArray=initialize.defineInitValues(patientIdDic,dataDir,my_fit)

parsFile=os.path.join(outDir,modelName,'parWins'+modelName+patientId+'.xlsx')
tmp=pd.read_excel(parsFile)
parDicAdd={'r_S':tmp['r_S'].iloc[0],'X0':tmp['X0'].iloc[0]}

tx = utils.ExtractTxsFromDf(dataDf,txCols=txCols)

testo=[]
if(boolTinput):
    nanT=np.isnan(dataDf['Testosterone'])
    if(nanT[0]):
        dataDf.loc[0,'Testosterone']=np.nanmax(dataDf['Testosterone'])
        nanT[0]=False
    dataDf['Tlog']=np.log10(dataDf['Testosterone'])
    f1T = scipy.interpolate.interp1d(dataDf.Days[~nanT],dataDf['Tlog'][~nanT],fill_value="extrapolate",kind="linear")#"extrapolate"
    teval=range(0,dataDf['Days'][len(dataDf['Days'])-1])
    testo=f1T(teval)
    testo=10**testo
    i=0
    tx=[]
    for d in teval:
        tx.append([d,d+1,dataDf['ADT'].loc[i],dataDf['Abi'].loc[i],dataDf['ARSI'].loc[i]])
        if(d>=dataDf['Days'].loc[i+1]):
            i=i+1 

topDf=aFuns.runSim(tmpModel,patientId,patInd,lArray,parDicAdd,tx,my_fit,boolTinput,testo,solver_kws)

#del tMax
#tMax=290

if(cols[pi]=='r'):lw=6
elif(cols[pi]=='b'):lw=4
else:lw=2

fig, ax = plt.subplots(1, 1)
for t in range(0,len(topDf)):
    if(topDf['ADT'].iloc[t]==1 and (topDf['Abi'].iloc[t]==1 or topDf['ARSI'].iloc[t]==1 )):
        col='m'
    elif(topDf['ADT'].iloc[t]==1 and topDf['Abi'].iloc[t]==0):
        col='b'
    else:
        col='k'
    if(thisPlot=='PSA'):
        plt.plot(topDf['Days'].iloc[t:t+2]/30,topDf['PSAi'].iloc[t:t+2],c=col,linewidth=15)
    else:
        plt.plot(topDf['Days'].iloc[t:t+2]/30,topDf['PSAi'].iloc[t:t+2]/topDf['T'].iloc[t:t+2],c=col,linewidth=15)
if(thisPlot=='PSA'):
    plt.scatter(dataDf['Days']/30,dataDf['PSA'],c=cols[pi],s=250,edgecolors='yellow',marker='D')
else:
    plt.scatter(dataDf['Days']/30,dataDf['PSA']/dataDf['Testosterone'],c=cols[pi],s=250,edgecolors='yellow',marker='D')

#plt.ylim([-1,33])
ax.set_xlim([-.2,7.8])
if(thisPlot=='PT'):
    ax.set_ylim([.0001,10])
else:
    ax.set_ylim([.01,60])
ax.set(yscale='log')